In [ ]:
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import numpy as np
import keras.backend as K
import os

IMG_SIZE = (224,224)
DATASET_NAME = "test_train_dataset"
SPLITTED_IMG_SET = Path(DATASET_NAME)
TRAIN_DIR = SPLITTED_IMG_SET / "train"
TEST_DIR = SPLITTED_IMG_SET /"test"
VAL_DIR = SPLITTED_IMG_SET / "val"
IMGS_CTGS = [d for d in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, d))]
BATCH_SIZE = 64
N_EPOCHS = 15
CLASS_MODE = "categorical"
ACTIVATION_FUNCTION = "softmax"
LOSS_FUNCTION = "categorical_crossentropy"
NOT_FINE_TUNED_MODEL_NAME = "recall_optm_not_fine_tuned_" + DATASET_NAME + ".hdf5"
FINE_TUNED_MODEL_NAME = "recall_optm_fine_tuned_" + DATASET_NAME + ".hdf5"
INPUT_SHAPE = (224, 224, 3)
optim_1 = Adam(learning_rate=0.001)
n_classes= len(IMGS_CTGS)

TRAIN = False

In [ ]:
class VGG16Model:
    def __init__(self) -> None:
        pass

    def recall(self, y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def create_model(
        self,
        input_shape,
        n_classes,
        optimizer="rmsprop",
        fine_tune=0,
        activation_func="sigmoid",
        loss_func="categorical_crossentropy",
    ):
        """
        Compiles a model integrated with VGG16 pretrained layers

        input_shape: tuple - the shape of input images (width, height, channels)
        n_classes: int - number of classes for the output layer
        optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
        fine_tune: int - The number of pre-trained layers to unfreeze.
                    If set to 0, all pretrained layers will freeze during training
        """

        # Pretrained convolutional layers are loaded using the Imagenet weights.
        # Include_top is set to False, in order to exclude the model's fully-connected layers.
        conv_base = VGG16(
            include_top=False, weights="imagenet", input_shape=input_shape
        )

        # Defines how many layers to freeze during training.
        # Layers in the convolutional base are switched from trainable to non-trainable
        # depending on the size of the fine-tuning parameter.
        if fine_tune > 0:
            for layer in conv_base.layers[:-fine_tune]:
                layer.trainable = False
        else:
            for layer in conv_base.layers:
                layer.trainable = False

        # Create a new 'top' of the model (i.e. fully-connected layers).
        # This is 'bootstrapping' a new top_model onto the pretrained layers.
        top_model = conv_base.output
        top_model = Flatten(name="flatten")(top_model)
        top_model = Dense(4096, activation="relu")(top_model)
        top_model = Dense(1072, activation="relu")(top_model)
        top_model = Dropout(0.2)(top_model)
        output_layer = Dense(n_classes, activation=activation_func)(top_model)

        # Group the convolutional base and new fully-connected layers into a Model object.
        model = Model(inputs=conv_base.input, outputs=output_layer)

        # Compiles the model for training.
        model.compile(optimizer=optimizer, loss=loss_func, metrics=[self.recall])
        return model

    

In [ ]:

train_generator = ImageDataGenerator(rotation_range=90, 
                                        brightness_range=[0.1, 0.7],
                                        width_shift_range=0.5, 
                                        height_shift_range=0.5,
                                        horizontal_flip=True, 
                                        vertical_flip=True,
                                        validation_split=0.15,
                                        preprocessing_function=preprocess_input)

test_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
if TRAIN:
    traingen = train_generator.flow_from_directory(TRAIN_DIR,
                                                   target_size=IMG_SIZE,
                                                   class_mode= CLASS_MODE,
                                                   classes=IMGS_CTGS,
                                                   subset='training',
                                                   batch_size=BATCH_SIZE, 
                                                   shuffle=True,
                                                   seed=42)

    validgen = train_generator.flow_from_directory(VAL_DIR,
                                                   target_size=IMG_SIZE,
                                                   class_mode=CLASS_MODE,
                                                   classes=IMGS_CTGS,
                                                   subset='validation',
                                                   batch_size=BATCH_SIZE,
                                                   shuffle=True,
                                                   seed=42)

    testgen = test_generator.flow_from_directory(TEST_DIR,
                                                    target_size=IMG_SIZE,
                                                    class_mode=CLASS_MODE,
                                                    classes=IMGS_CTGS,
                                                    batch_size=1,
                                                    shuffle=False,
                                                    seed=42)

In [ ]:
if TRAIN:
    input_shape = (224, 224, 3)
    optim_1 = Adam(learning_rate=0.001)
    n_classes= len(IMGS_CTGS)

    n_steps = traingen.samples // BATCH_SIZE
    n_val_steps = validgen.samples // BATCH_SIZE
    n_epochs = N_EPOCHS

    # First we'll train the model without Fine-tuning
vgg_model = VGG16Model().create_model(input_shape, n_classes, optim_1, fine_tune=0, activation_func = ACTIVATION_FUNCTION, loss_func = LOSS_FUNCTION)

In [ ]:
if TRAIN:
    plot_loss_1 = PlotLossesCallback()

    # ModelCheckpoint callback - save best weights
    tl_checkpoint_1 = ModelCheckpoint(filepath=NOT_FINE_TUNED_MODEL_NAME,
                                      save_best_only=True,
                                      verbose=1)

    # EarlyStopping
    early_stop = EarlyStopping(monitor='val_loss',
                               patience=10,
                               restore_best_weights=True,
                               mode='min')

In [ ]:
if TRAIN:
    vgg_history = vgg_model.fit(traingen,
                                batch_size=BATCH_SIZE,
                                epochs=n_epochs,
                                validation_data=validgen,
                                steps_per_epoch=n_steps,
                                validation_steps=n_val_steps,
                                callbacks=[tl_checkpoint_1, early_stop, plot_loss_1],
                                verbose=1)

In [ ]:
if TRAIN:
    vgg_model.load_weights(NOT_FINE_TUNED_MODEL_NAME) # initialize the best trained weights

    true_classes = testgen.classes
    class_indices = traingen.class_indices
    class_indices = dict((v,k) for k,v in class_indices.items())

    vgg_preds = vgg_model.predict(testgen)
    vgg_pred_classes = np.argmax(vgg_preds, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

if TRAIN:
    vgg_acc = accuracy_score(true_classes, vgg_pred_classes)
    print("VGG16 Model Accuracy without Fine-Tuning: {:.2f}%".format(vgg_acc * 100))

In [ ]:

'''if TRAIN:
    traingen.reset()
    validgen.reset()
    testgen.reset()

    # Use a smaller learning rate
    optim_2 = Adam(lr=0.0001)
    tl_checkpoint_1 = ModelCheckpoint(filepath=FINE_TUNED_MODEL_NAME,
                                      save_best_only=True,
                                      verbose=1)


    # Re-compile the model, this time leaving the last 2 layers unfrozen for Fine-Tuning
vgg_model_ft = VGG16Model().create_model(input_shape, n_classes, optim_1, fine_tune=1, activation_func = ACTIVATION_FUNCTION, loss_func = LOSS_FUNCTION)'''

In [ ]:
'''%%time
if TRAIN:


    plot_loss_2 = PlotLossesCallback()

    # Retrain model with fine-tuning
    vgg_ft_history = vgg_model_ft.fit(traingen,
                                      batch_size=BATCH_SIZE,
                                      epochs=n_epochs,
                                      validation_data=validgen,
                                      steps_per_epoch=n_steps, 
                                      validation_steps=n_val_steps,
                                      callbacks=[tl_checkpoint_1, early_stop, plot_loss_2],
                                      verbose=1)'''

In [ ]:
'''if TRAIN:
    # Generate predictions
    vgg_model_ft.load_weights(FINE_TUNED_MODEL_NAME) # initialize the best trained weights

    vgg_preds_ft = vgg_model_ft.predict(testgen)
    vgg_pred_classes_ft = np.argmax(vgg_preds_ft, axis=1)'''

In [ ]:
'''if TRAIN:
    vgg_acc_ft = accuracy_score(true_classes, vgg_pred_classes_ft)
    print(vgg_pred_classes_ft)

    print("VGG16 Model Accuracy with Fine-Tuning: {:.2f}%".format(vgg_acc_ft * 100))'''